# Experiments

This Jupyter Notebook presents different experiments done to videos. This experiments focus on the temporal consistency in cartoon-like videos. We focus on using the optical flow to see how consecutive frames change.

First, we import the functions needed from the notebook called "Functions".

In [20]:
#%run Functions.ipynb

In [13]:
import import_ipynb
from Functions import *

We load a video and store its frames in a list to then add occlusions to 3 consecutive frames.

In [2]:
cap = open_vid('VDB\L.mp4')
Frames = get_frames(cap)
_,_,fps,_ = get_props(cap)
F = Frames.copy()
loc=[[150,50]]
sizes = [[50,100]]
N = 15
F[N] = occlusions(F[N],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
F[N+1] = occlusions(F[N+1],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
F[N+2] = occlusions(F[N+2],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))

Width:  640
Height:  360
FPS:  23.976023976023978
Frame Count:  2326


We play the video to see the occlusions.<br>
Use 'q' to stop and close window at any time.

In [3]:
play_frames(F[:30],fps)

Delay:  41


Then we calculate different properties in the series of frames.

First, we calculate and display the magnitude of the optical flow by using Furneback's method.<br>
Use the arrows to move forward or backward in the frame sequence.<br>
Use 'q' at any time to stop and close windows.

In [8]:
_ = OFM(F[:30])

We can also see the vector field of the optical flow in with the green arrows. Also, we see the sum of these vectors as the red arrow.

In [6]:
_ = OFV(F[:30])

We can also use the Lucas-Kanade method to calculate the optical flow.

In [7]:
_ = OFLK(F[:30])

And finally, we can compute the optical flow using Phase Correlation.

In [9]:
_ = PhaseC(F[:30])

Another way to se how consecutive frames change from one to another, we can calculate its difference.

First, we calculate the difference between a pair of consecutive frames and use a threshold to create a binary image that shows the region with most changes.

In [10]:
_ = frame_dif(F[:30])

But for more simplicity, we can just calculate the differences and normalize them.

In [11]:
_ = frame_dif1(F[:30])

Now, lets try to cartoonize an image.

In [2]:
file = 'IDB/therock.jpg'
img = cv2.imread(file)
toon = cartoonize_image(img,kc=3)
display_images([img,toon])

What about cartoonizing a video?<br>
To speed up the cartoonization, the frames in the video are resized to smaller ones and then returned to their original size. Take in consideration that this will also degrade the quality of the cartoonized video.

In [35]:
cap = open_vid('VDB/U.mp4')
Frames = get_frames(cap)
w,h,fps,_ = get_props(cap)
Toon = [cv2.resize(cartoonize_image(cv2.resize(f,(200,200),interpolation=cv2.INTER_AREA)),(w,h),interpolation=cv2.INTER_CUBIC) for f in Frames]
#save_vid(Toon,'Cartoonized/U_toonzz.mp4',fps)

Width:  640
Height:  320
FPS:  24.0
Frame Count:  69


Lets see the original video.

In [33]:
play_frames(Frames,fps)

Delay:  41


Lets see the cartoonized video now.

In [36]:
play_frames(Toon,fps)

Delay:  41


Now lets see how the optical flow behave for the original video and the cartoonized video.

In [4]:
cap = open_vid('VDB/U.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
Frames = get_frames(cap)
cap = open_vid('Cartoonized/U_toon.mp4')
Toon = get_frames(cap)
play_frames(Frames,fps)
play_frames(Toon,fps)
_ = OFV(Frames,20)
_ = OFV(Toon,20)

Delay:  41
Delay:  41


To have more control over the optical flow, lets capture the video frames from the webcam and display it in real time.

In [11]:
cap_of()

We can compute different variability indexes of the pixel of an image. Some of these are calculated below as the relative luminance, the color variace and a variability index based on neighboring pixels. The greater the variability index, the more variability there is.

In [10]:
file = 'IDB/party.jpg'
img = cv2.imread(file)
print(relative_luminance(img))
print(color_variance(img))
print(calculate_variability(img))
display_frame(img)

0.5007049189404322
0.003574134719467966
0.7609107919731268


In [5]:
#Entorno/Interfaz

## Temporal Gradient

In [17]:
cap = open_vid('Cartoonized/U_toon.mp4')
Frames = get_frames(cap)
delete_png_files('saved_frames')
save_frames(Frames,'saved_frames')

In [23]:
import cv2
import numpy as np

def calculate_and_display_gradients(img1, img2):
    # Calculate the spatial gradients using Sobel operator
    grad_x1 = cv2.Sobel(img1, cv2.CV_64F, 1, 0, ksize=5)
    grad_y1 = cv2.Sobel(img1, cv2.CV_64F, 0, 1, ksize=5)

    grad_x2 = cv2.Sobel(img2, cv2.CV_64F, 1, 0, ksize=5)
    grad_y2 = cv2.Sobel(img2, cv2.CV_64F, 0, 1, ksize=5)

    # Calculate the temporal gradient
    temporal_gradient = cv2.absdiff(img2, img1)

    # Normalize gradients for display purposes
    grad_x1_display = cv2.convertScaleAbs(grad_x1)
    grad_y1_display = cv2.convertScaleAbs(grad_y1)
    grad_x2_display = cv2.convertScaleAbs(grad_x2)
    grad_y2_display = cv2.convertScaleAbs(grad_y2)
    temporal_gradient_display = cv2.convertScaleAbs(temporal_gradient)

    return grad_x1, grad_y1, grad_x2, grad_y2, temporal_gradient, grad_x1_display, grad_y1_display, grad_x2_display, grad_y2_display, temporal_gradient_display

def calculate_and_display_movement(img1_path, img2_path):
    # Read the two consecutive images
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        raise ValueError("One or both of the images could not be loaded. Check the file paths.")

    # Calculate gradients
    grad_x1, grad_y1, grad_x2, grad_y2, temporal_gradient, grad_x1_display, grad_y1_display, grad_x2_display, grad_y2_display, temporal_gradient_display = calculate_and_display_gradients(img1, img2)

    # Calculate the magnitude and angle of optical flow
    magnitude = cv2.magnitude(grad_x2, grad_y2)
    angle = cv2.phase(grad_x2, grad_y2, angleInDegrees=True)

    # Normalize magnitude for display purposes
    magnitude_display = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude_display = cv2.convertScaleAbs(magnitude_display)

    # Display the results
    cv2.imshow("Image 1", img1)
    cv2.imshow("Image 2", img2)
    #cv2.imshow("Gradient X - Image 1", grad_x1_display)
    #cv2.imshow("Gradient Y - Image 1", grad_y1_display)
    #cv2.imshow("Gradient X - Image 2", grad_x2_display)
    #cv2.imshow("Gradient Y - Image 2", grad_y2_display)
    cv2.imshow("Temporal Gradient", temporal_gradient_display)
    cv2.imshow("Optical Flow Magnitude", magnitude_display)

    # Optional: Visualize flow direction using HSV color space
    hsv = np.zeros((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)
    hsv[..., 1] = 255
    hsv[..., 0] = angle / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    optical_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow("Optical Flow Direction", optical_flow)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
calculate_and_display_movement('saved_frames/frame_000.png', 'saved_frames/frame_001.png')


In [24]:
import cv2
import numpy as np

def calculate_and_display_differences(image1_path, image2_path):
    # Read the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check the file paths.")

    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Calculate the absolute difference between the images
    diff_rgb = np.abs(img1_rgb - img2_rgb)
    diff_gray = np.abs(img1_gray.astype(np.float32) - img2_gray.astype(np.float32))

    # Normalize the differences to [0, 255]
    norm_diff_rgb = cv2.normalize(diff_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_diff_gray = cv2.normalize(diff_gray, None, 0, 255, cv2.NORM_MINMAX)
    
    # Split the RGB difference into color channels
    diff_r, diff_g, diff_b = cv2.split(norm_diff_rgb)
    
    # Convert the single channel differences back to 3-channel images for display
    diff_r_colored = cv2.merge([diff_r, np.zeros_like(diff_r), np.zeros_like(diff_r)])
    diff_g_colored = cv2.merge([np.zeros_like(diff_g), diff_g, np.zeros_like(diff_g)])
    diff_b_colored = cv2.merge([np.zeros_like(diff_b), np.zeros_like(diff_b), diff_b])
    
    # Display the original images and differences
    cv2.imshow('Original Image 1', img1)
    cv2.imshow('Original Image 2', img2)
    cv2.imshow('Normalized Difference - Red Channel', diff_r_colored)
    cv2.imshow('Normalized Difference - Green Channel', diff_g_colored)
    cv2.imshow('Normalized Difference - Blue Channel', diff_b_colored)
    cv2.imshow('Normalized Grayscale Difference', norm_diff_gray.astype(np.uint8))
    
    # Wait until a key is pressed and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
calculate_and_display_differences('saved_frames/frame_000.png', 'saved_frames/frame_001.png')


In [25]:
import cv2
import numpy as np

def calculate_temporal_gradient(image1_path, image2_path):
    # Read the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check the file paths.")

    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    
    # Convert images to grayscale
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Calculate the temporal gradient (difference) between the images
    grad_rgb = np.abs(img2_rgb - img1_rgb)
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    # Normalize the gradients to [0, 255]
    norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
    
    # Split the RGB gradient into color channels
    grad_r, grad_g, grad_b = cv2.split(norm_grad_rgb)
    
    # Convert the single channel gradients back to 3-channel images for display
    grad_r_colored = cv2.merge([grad_r, np.zeros_like(grad_r), np.zeros_like(grad_r)])
    grad_g_colored = cv2.merge([np.zeros_like(grad_g), grad_g, np.zeros_like(grad_g)])
    grad_b_colored = cv2.merge([np.zeros_like(grad_b), np.zeros_like(grad_b), grad_b])
    
    # Display the gradients
    cv2.imshow('Original Image 1', img1)
    cv2.imshow('Original Image 2', img2)
    cv2.imshow('Temporal Gradient - Red Channel', grad_r_colored)
    cv2.imshow('Temporal Gradient - Green Channel', grad_g_colored)
    cv2.imshow('Temporal Gradient - Blue Channel', grad_b_colored)
    cv2.imshow('Temporal Gradient - Grayscale', norm_grad_gray.astype(np.uint8))
    
    # Wait until a key is pressed and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
calculate_temporal_gradient('saved_frames/frame_000.png', 'saved_frames/frame_001.png')


In [31]:
import cv2
import numpy as np

def calculate_temporal_consistency(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < 2:
        raise ValueError("Video must have at least two frames to calculate temporal consistency.")
    
    # Initialize variables to accumulate error values
    total_error_rgb = np.zeros(3)  # R, G, B channels
    total_error_gray = 0
    num_frames = 0

    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    while True:
        # Read the next frame
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        
        # Compute the absolute difference between consecutive frames
        diff_rgb = np.abs(curr_frame_rgb.astype(np.float32) - prev_frame_rgb.astype(np.float32))
        diff_gray = np.abs(curr_frame_gray.astype(np.float32) - prev_frame_gray.astype(np.float32))
        
        # Calculate the mean absolute error for the current frame pair
        mean_error_rgb = np.mean(diff_rgb, axis=(0, 1))  # Mean error for R, G, B channels
        mean_error_gray = np.mean(diff_gray)
        
        total_error_rgb += mean_error_rgb
        total_error_gray += mean_error_gray
        num_frames += 1
        
        # Update previous frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Compute the average temporal consistency index
    if num_frames == 0:
        raise ValueError("No frames processed. Check the video file.")
    
    temporal_consistency_index_rgb = total_error_rgb / num_frames
    temporal_consistency_index_gray = total_error_gray / num_frames
    
    # Release video capture object
    cap.release()
    
    return temporal_consistency_index_rgb, temporal_consistency_index_gray

# Example usage
video_path = 'Cartoonized/U_toonzz.mp4'
temporal_consistency_index_rgb, temporal_consistency_index_gray = calculate_temporal_consistency(video_path)
print(f'Temporal Consistency Index (RGB): R: {temporal_consistency_index_rgb[0]}, G: {temporal_consistency_index_rgb[1]}, B: {temporal_consistency_index_rgb[2]}')
print(f'Temporal Consistency Index (Grayscale): {temporal_consistency_index_gray}')


Temporal Consistency Index (RGB): R: 13.011466811685, G: 12.35071282527026, B: 12.59720345104442
Temporal Consistency Index (Grayscale): 12.210450228522806


In [32]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_similarity_indexes(image1_path, image2_path):
    # Read the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check the file paths.")
    
    # Convert images to RGB (OpenCV loads images in BGR format by default)
    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    # Convert images to grayscale
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))
    
    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    # Compute histogram correlation for each channel
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

# Example usage
image1_path = 'saved_frames/frame_000.png'
image2_path = 'saved_frames/frame_001.png'
similarity_indexes = calculate_similarity_indexes(image1_path, image2_path)
print(f'Similarity Indexes: {similarity_indexes}')


Similarity Indexes: {'MSE_RGB': 56.19447428385417, 'SSIM_Gray': 0.7325608369867386, 'Hist_Corr_R': 0.16422995202312327, 'Hist_Corr_G': 0.2144371040869527, 'Hist_Corr_B': 0.3156782404000596, 'Edge_Similarity': 0.9294189453125}


In [33]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def identify_abnormality(image1_path, image2_path):
    # Read the images
    img1 = cv2.imread(image1_path)
    img2 = cv2.imread(image2_path)
    
    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check the file paths.")
    
    # Calculate temporal gradients
    grad_rgb, grad_gray = calculate_temporal_gradient(img1, img2)
    
    # Calculate similarity indexes
    similarity_indexes = calculate_similarity_indexes(img1, img2)
    
    # Combine gradients and similarity indexes to identify abnormalities
    # Normalize gradients for visibility
    norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
    norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
    
    # Create a mask where significant changes or low similarity occur
    threshold = 50  # Set an appropriate threshold value
    abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
    abnormal_gray_mask = norm_grad_gray > threshold

    # Highlight the regions of interest
    highlighted_rgb = img1.copy()
    highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
    highlighted_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
    
    # Display the results
    cv2.imshow('Original Image 1', img1)
    cv2.imshow('Original Image 2', img2)
    cv2.imshow('Temporal Gradient (RGB)', norm_grad_rgb.astype(np.uint8))
    cv2.imshow('Temporal Gradient (Gray)', norm_grad_gray.astype(np.uint8))
    cv2.imshow('Abnormal Regions (RGB)', highlighted_rgb)
    cv2.imshow('Abnormal Regions (Gray)', highlighted_gray)
    
    # Print similarity indexes
    print(f'Similarity Indexes: {similarity_indexes}')
    
    # Wait until a key is pressed and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image1_path = 'saved_frames/frame_000.png'
image2_path = 'saved_frames/frame_001.png'
identify_abnormality(image1_path, image2_path)


Similarity Indexes: {'MSE_RGB': 56.19447428385417, 'SSIM_Gray': 0.7325608369867386, 'Hist_Corr_R': 0.16422995202312327, 'Hist_Corr_G': 0.2144371040869527, 'Hist_Corr_B': 0.3156782404000596, 'Edge_Similarity': 0.9294189453125}


In [36]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4 codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * 2, frame_height * 2))  # Updated size for combined frames
    
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = rgb2gray(prev_frame_rgb)
    
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = rgb2gray(curr_frame_rgb)
        
        # Calculate temporal gradients and similarity indexes
        grad_rgb, grad_gray = calculate_temporal_gradient(prev_frame, curr_frame)
        similarity_indexes = calculate_similarity_indexes(prev_frame, curr_frame)
        
        # Normalize gradients for visibility
        norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX)
        norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX)
        
        # Create masks for abnormalities
        threshold = 50  # Set an appropriate threshold value
        abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
        abnormal_gray_mask = norm_grad_gray > threshold

        # Highlight the regions of interest
        highlighted_rgb = curr_frame.copy()
        highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
        highlighted_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
        
        # Combine the results into a single frame
        combined_frame = np.hstack((curr_frame, cv2.cvtColor(highlighted_rgb, cv2.COLOR_BGR2RGB)))
        combined_frame = np.vstack((combined_frame, np.hstack((cv2.cvtColor(highlighted_gray, cv2.COLOR_GRAY2BGR), norm_grad_rgb.astype(np.uint8)))))
        
        # Write the combined frame to the output video
        out.write(combined_frame)
        
        # Update previous frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'Cartoonized/U_toon.mp4'
output_path = 'output_abnormalities.mp4'
process_video(video_path, output_path)


In [41]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2gray

def calculate_temporal_gradient(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)

    # Calculate the temporal gradient (difference) for each channel and grayscale
    grad_rgb = np.abs(img2_rgb.astype(np.float32) - img1_rgb.astype(np.float32))
    grad_gray = np.abs(img2_gray.astype(np.float32) - img1_gray.astype(np.float32))

    return grad_rgb, grad_gray

def calculate_similarity_indexes(image1, image2):
    # Convert images to RGB and grayscale
    img1_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    img1_gray = rgb2gray(img1_rgb)
    img2_gray = rgb2gray(img2_rgb)
    
    # Resize images to the same size if they are different
    if img1_rgb.shape != img2_rgb.shape:
        img2_rgb = cv2.resize(img2_rgb, (img1_rgb.shape[1], img1_rgb.shape[0]))
        img2_gray = cv2.resize(img2_gray, (img1_gray.shape[1], img1_gray.shape[0]))

    # Compute Mean Squared Error (MSE) for RGB
    mse_rgb = np.mean((img1_rgb - img2_rgb) ** 2)
    
    # Compute Structural Similarity Index (SSIM) for grayscale
    ssim_gray, _ = ssim(img1_gray, img2_gray, full=True)
    
    # Compute Color Histograms for RGB
    hist1_r = cv2.calcHist([img1_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist2_r = cv2.calcHist([img2_rgb[:, :, 0]], [0], None, [256], [0, 256])
    hist1_g = cv2.calcHist([img1_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist2_g = cv2.calcHist([img2_rgb[:, :, 1]], [0], None, [256], [0, 256])
    hist1_b = cv2.calcHist([img1_rgb[:, :, 2]], [0], None, [256], [0, 256])
    hist2_b = cv2.calcHist([img2_rgb[:, :, 2]], [0], None, [256], [0, 256])
    
    hist_corr_r = cv2.compareHist(hist1_r, hist2_r, cv2.HISTCMP_CORREL)
    hist_corr_g = cv2.compareHist(hist1_g, hist2_g, cv2.HISTCMP_CORREL)
    hist_corr_b = cv2.compareHist(hist1_b, hist2_b, cv2.HISTCMP_CORREL)
    
    # Compute Edge-based similarity (using Canny edge detector)
    edges1 = cv2.Canny(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY), 100, 200)
    edges2 = cv2.Canny(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY), 100, 200)
    
    edge_similarity = np.sum(edges1 == edges2) / np.size(edges1)
    
    return {
        'MSE_RGB': mse_rgb,
        'SSIM_Gray': ssim_gray,
        'Hist_Corr_R': hist_corr_r,
        'Hist_Corr_G': hist_corr_g,
        'Hist_Corr_B': hist_corr_b,
        'Edge_Similarity': edge_similarity
    }

def add_label(image, text, position=(10, 30), font_scale=1, color=(255, 255, 255), thickness=2):
    """Add a label to an image."""
    return cv2.putText(image, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)

def process_video(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file. Check the file path.")
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using MP4 codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * 3, frame_height * 2))  # Updated size for combined frames
    
    ret, prev_frame = cap.read()
    if not ret:
        raise ValueError("Failed to read the first frame.")
    
    prev_frame_rgb = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
    prev_frame_gray = rgb2gray(prev_frame_rgb)
    
    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break
        
        curr_frame_rgb = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2RGB)
        curr_frame_gray = rgb2gray(curr_frame_rgb)
        
        # Calculate temporal gradients and similarity indexes
        grad_rgb, grad_gray = calculate_temporal_gradient(prev_frame, curr_frame)
        similarity_indexes = calculate_similarity_indexes(prev_frame, curr_frame)
        
        # Normalize gradients for visibility
        norm_grad_rgb = cv2.normalize(grad_rgb, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        norm_grad_gray = cv2.normalize(grad_gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        
        # Convert single-channel grayscale to 3-channel RGB for displaying
        norm_grad_gray_rgb = cv2.cvtColor(norm_grad_gray, cv2.COLOR_GRAY2RGB)
        
        # Create masks for abnormalities
        threshold = 50  # Set an appropriate threshold value
        abnormal_rgb_mask = np.max(norm_grad_rgb, axis=2) > threshold
        abnormal_gray_mask = norm_grad_gray > threshold

        # Highlight the regions of interest
        highlighted_rgb = curr_frame.copy()
        highlighted_rgb[abnormal_rgb_mask] = [0, 0, 255]  # Mark changes in red
        highlighted_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        highlighted_gray[abnormal_gray_mask] = 255  # Mark changes in white
        
        # Create the combined frame
        combined_frame = np.zeros((frame_height * 2, frame_width * 3, 3), dtype=np.uint8)
        combined_frame[0:frame_height, 0:frame_width] = prev_frame
        combined_frame[0:frame_height, frame_width:frame_width*2] = curr_frame
        combined_frame[0:frame_height, frame_width*2:frame_width*3] = highlighted_rgb
        combined_frame[frame_height:frame_height*2, 0:frame_width] = cv2.cvtColor(highlighted_gray, cv2.COLOR_GRAY2BGR)
        combined_frame[frame_height:frame_height*2, frame_width:frame_width*2] = norm_grad_rgb
        combined_frame[frame_height:frame_height*2, frame_width*2:frame_width*3] = norm_grad_gray_rgb
        
        # Add labels to the combined frame
        combined_frame = add_label(combined_frame, "Previous Frame", position=(10, 30))
        combined_frame = add_label(combined_frame, "Current Frame", position=(frame_width + 10, 30))
        combined_frame = add_label(combined_frame, "Highlighted Changes", position=(frame_width * 2 + 10, 30))
        combined_frame = add_label(combined_frame, "Gray Gradient", position=(10, frame_height + 30))
        combined_frame = add_label(combined_frame, "RGB Gradient", position=(frame_width + 10, frame_height + 30))
        combined_frame = add_label(combined_frame, "Gray Gradient (Norm)", position=(frame_width * 2 + 10, frame_height + 30))
        
        # Write the combined frame to the output video
        out.write(combined_frame)
        
        # Update previous frame
        prev_frame = curr_frame
        prev_frame_rgb = curr_frame_rgb
        prev_frame_gray = curr_frame_gray
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'Cartoonized/U_toon.mp4'
output_path = 'output_abnormalities1.mp4'
process_video(video_path, output_path)
